## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://services.arcgis.com/UHg8l1wC48WQyDSO/arcgis/rest/services/reportareadb_view/FeatureServer/0//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=Report_Are%2C+Case_Count%2Csurvey_date&returnGeometry=true&returnCentroid=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [7]:
r = requests.get(url)

In [8]:
data = r.json()

In [9]:
data

{'error': {'code': 499,
  'message': 'Token Required',
  'messageCode': 'GWM_0003',
  'details': ['Token Required']}}

## Parse

In [10]:
dict_list = []

In [11]:
for item in data["features"]:
    timestamp = item["attributes"]["survey_date"]
    timestamp = datetime.fromtimestamp((timestamp / 1000))
    d = dict(
        county="El Dorado",
        area=item["attributes"]["Report_Are"],
        confirmed_cases=item["attributes"]["Case_Count"],
        county_date=timestamp,
    )
    dict_list.append(d)

KeyError: 'features'

Convert to dataframe

In [ ]:
df = pd.DataFrame(dict_list)

In [ ]:
df["county_date"] = df["county_date"].dt.strftime("%Y-%m-%d")

Set the date

In [ ]:
tz = pytz.timezone("America/Los_Angeles")

In [ ]:
today = datetime.now(tz).date()

In [ ]:
slug = "el-dorado"

## Vet

In [ ]:
try:
    assert not len(df) > 8
except AssertionError:
    raise AssertionError("El Dorado's area scraper has extra rows")

In [ ]:
try:
    assert not len(df) < 8
except AssertionError:
    raise AssertionError("El Dorado's area scraper is missing rows")

## Export

In [ ]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [ ]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [ ]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [ ]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [ ]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)